## Import modules and environments

In [20]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import sys

import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm as _tqdm

import random
import time
from collections import defaultdict
import operator

def tqdm(*args, **kwargs):
    return _tqdm(*args, **kwargs, mininterval=1)  # Safety, do not overflow buffer

EPS = float(np.finfo(np.float32).eps)

assert sys.version_info[:3] >= (3, 6, 0), "Make sure you have Python 3.6 installed!"

In [26]:
import gym
env_cartpole = gym.envs.make("CartPole-v0") # Has two actions, See doc for mor info, ??env_cartpole.env
env_car = gym.envs.make("MountainCar-v0")   # Has three actions
env_pen = gym.envs.make("Pendulum-v0")      # Has continious action values like array([-1.2552279] or array([1.7774895] 
env_acrobot = gym.envs.make("Acrobot-v1")   # Has Three actions, applying +1, 0 or -1 torque on the joint between #
                                            # the two pendulum links.   See doc for more info, ??env_acrobot.env

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [83]:
#??env_acrobot.env
#env_acrobot.action_space.sample()
#env = env_cartpole
#env = env_car
env = env_acrobot

In [54]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"  
#device = "cpu"
print(device)

cpu


In [55]:

name = env.unwrapped.spec.id
print(name)

MountainCar-v0


### Test demo environment

In [94]:
import time
# The nice thing about the CARTPOLE is that it has very nice rendering functionality (if you are on a local environment). Let's have a look at an episode
obs = env.reset()
env.render()
done = False
while not done:
    obs, reward, done, _ = env.step(env.action_space.sample())
    env.render()
    time.sleep(0.05)
env.close()  # Close the environment or you will have a lot of render screens soon

[ 0.05459818 -0.99850841 -0.87534148 -0.48350522  1.68543313  0.57962891]
500


### Our DQN model

In [6]:
class QNetwork(nn.Module):
    
    def __init__(self, device,num_input=4,num_hidden=128,num_output=2):
        nn.Module.__init__(self)
        self.device = device
        self.l1 = nn.Linear(num_input, num_hidden)
        self.l2 = nn.Linear(num_hidden, num_output)
        self.input_dim = num_input
        self.hidden_dim = num_hidden
        self.output_dim = num_output

    def forward(self, x):
        if not torch.is_tensor(x):
            x = torch.Tensor(x)
            
        x = x.to(self.device)
        
        forward_pass = nn.Sequential(
            self.l1,
            nn.ReLU(),
            self.l2
        )
        
        return forward_pass(x)

### Epsilon ($\epsilon$) greedy policy

In [7]:
def get_epsilon(it):
    # YOUR CODE HERE
    epsilon = 1 - (min(it,1000) * 0.00095) #After 1000 iterations epsilon should be 0.05
    
    return epsilon

In [8]:
def parameter_schedule(it,max_iter,initial_value,final_value):
    #Following general formula of get epislon
    parameter_value = initial_value - (min(it,max_iter) * ( (initial_value - final_value) / max_iter) )
                            
    return parameter_value

In [9]:
def select_action(model, state, epsilon):
    # YOUR CODE HERE
    with torch.no_grad():
        Q_approx = model(state)
        a = int(np.random.rand() * model.output_dim) if np.random.rand() < epsilon else torch.argmax(Q_approx).item()
        
    return a

### Experience replays types

#### Uniform

In [10]:
class ReplayMemory:
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.name = "uniform"

    def push(self, transition):
        
        if len(self.memory) == self.capacity:
            self.memory = self.memory[1:]   
        
        self.memory.append(transition)

    def sample(self, batch_size):
        return random.sample(self.memory,batch_size)

    def __len__(self):
        return len(self.memory)

#### Prioritized Replay

In [11]:
from segment_tree import SumSegmentTree, MinSegmentTree

#Source OpenAI: https://github.com/openai/baselines/blob/master/baselines/deepq/replay_buffer.py

class ReplayBuffer(object):
    def __init__(self, size):
        """Create Replay buffer.
        Parameters
        ----------
        size: int
            Max number of transitions to store in the buffer. When the buffer
            overflows the old memories are dropped.
        """
        self._storage = []
        self._maxsize = size
        self._next_idx = 0

    def __len__(self):
        return len(self._storage)

    def add(self, obs_t, action, reward, obs_tp1, done):
        data = (obs_t, action, reward, obs_tp1, done)

        if self._next_idx >= len(self._storage):
            self._storage.append(data)
        else:
            self._storage[self._next_idx] = data
        self._next_idx = (self._next_idx + 1) % self._maxsize

    def _encode_sample(self, idxes):
        obses_t, actions, rewards, obses_tp1, dones = [], [], [], [], []
        for i in idxes:
            data = self._storage[i]
            obs_t, action, reward, obs_tp1, done = data
            obses_t.append(np.array(obs_t, copy=False))
            actions.append(np.array(action, copy=False))
            rewards.append(reward)
            obses_tp1.append(np.array(obs_tp1, copy=False))
            dones.append(done)
        return np.array(obses_t), np.array(actions), np.array(rewards), np.array(obses_tp1), np.array(dones)

    def sample(self, batch_size):
        """Sample a batch of experiences.
        Parameters
        ----------
        batch_size: int
            How many transitions to sample.
        Returns
        -------
        obs_batch: np.array
            batch of observations
        act_batch: np.array
            batch of actions executed given obs_batch
        rew_batch: np.array
            rewards received as results of executing act_batch
        next_obs_batch: np.array
            next set of observations seen after executing act_batch
        done_mask: np.array
            done_mask[i] = 1 if executing act_batch[i] resulted in
            the end of an episode and 0 otherwise.
        """
        idxes = [random.randint(0, len(self._storage) - 1) for _ in range(batch_size)]
        return self._encode_sample(idxes)


class PrioritizedReplayBuffer(ReplayBuffer):
    def __init__(self, size, alpha):
        """Create Prioritized Replay buffer.
        Parameters
        ----------
        size: int
            Max number of transitions to store in the buffer. When the buffer
            overflows the old memories are dropped.
        alpha: float
            how much prioritization is used
            (0 - no prioritization, 1 - full prioritization)
        See Also
        --------
        ReplayBuffer.__init__
        """
        super(PrioritizedReplayBuffer, self).__init__(size)
        assert alpha >= 0
        self._alpha = alpha

        it_capacity = 1
        while it_capacity < size:
            it_capacity *= 2

        self._it_sum = SumSegmentTree(it_capacity)
        self._it_min = MinSegmentTree(it_capacity)
        self._max_priority = 1.0
        self.name = "prioritized"

    def add(self, *args, **kwargs):
        """See ReplayBuffer.store_effect"""
        idx = self._next_idx
        super().add(*args, **kwargs)
        self._it_sum[idx] = self._max_priority ** self._alpha
        self._it_min[idx] = self._max_priority ** self._alpha

    def _sample_proportional(self, batch_size):
        res = []
        p_total = self._it_sum.sum(0, len(self._storage) - 1)
        every_range_len = p_total / batch_size
        for i in range(batch_size):
            mass = random.random() * every_range_len + i * every_range_len
            idx = self._it_sum.find_prefixsum_idx(mass)
            res.append(idx)
        return res

    def sample(self, batch_size, beta):
        """Sample a batch of experiences.
        compared to ReplayBuffer.sample
        it also returns importance weights and idxes
        of sampled experiences.
        Parameters
        ----------
        batch_size: int
            How many transitions to sample.
        beta: float
            To what degree to use importance weights
            (0 - no corrections, 1 - full correction)
        Returns
        -------
        obs_batch: np.array
            batch of observations
        act_batch: np.array
            batch of actions executed given obs_batch
        rew_batch: np.array
            rewards received as results of executing act_batch
        next_obs_batch: np.array
            next set of observations seen after executing act_batch
        done_mask: np.array
            done_mask[i] = 1 if executing act_batch[i] resulted in
            the end of an episode and 0 otherwise.
        weights: np.array
            Array of shape (batch_size,) and dtype np.float32
            denoting importance weight of each sampled transition
        idxes: np.array
            Array of shape (batch_size,) and dtype np.int32
            idexes in buffer of sampled experiences
        """
        assert beta > 0

        idxes = self._sample_proportional(batch_size)

        weights = []
        p_min = self._it_min.min() / self._it_sum.sum()
        max_weight = (p_min * len(self._storage)) ** (-beta)

        for idx in idxes:
            p_sample = self._it_sum[idx] / self._it_sum.sum()
            weight = (p_sample * len(self._storage)) ** (-beta)
            weights.append(weight / max_weight)
        weights = np.array(weights)
        encoded_sample = self._encode_sample(idxes)
        return tuple(list(encoded_sample) + [weights, idxes])

    def update_priorities(self, idxes, priorities):
        """Update priorities of sampled transitions.
        sets priority of transition at index idxes[i] in buffer
        to priorities[i].
        Parameters
        ----------
        idxes: [int]
            List of idxes of sampled transitions
        priorities: [float]
            List of updated priorities corresponding to
            transitions at the sampled idxes denoted by
            variable `idxes`.
        """
        assert len(idxes) == len(priorities)
        for idx, priority in zip(idxes, priorities):
            assert priority > 0
            assert 0 <= idx < len(self._storage)
            self._it_sum[idx] = priority ** self._alpha
            self._it_min[idx] = priority ** self._alpha

            self._max_priority = max(self._max_priority, priority)
            
def train_prioritized(model, memory, optimizer, batch_size, discount_factor,beta,prioritized_replay_eps=1e-6):
    # DO NOT MODIFY THIS FUNCTION
    
    # don't learn without some decent experience
    if len(memory) < batch_size:
        return None

    # random transition batch is taken from experience replay memory
    #----------------------------#Adjusted of normal train--------------------------------------------------------
    transitions = memory.sample(batch_size, beta=beta)
    (state, action, reward, next_state, done, weights, batch_idxes) = transitions#zip(*transitions)
    
    # convert to PyTorch and define types
    state = torch.tensor(state, dtype=torch.float, device = model.device)
    action = torch.tensor(action, dtype=torch.int64, device = model.device)  # Need 64 bit to use them as index
    next_state = torch.tensor(next_state, dtype=torch.float, device = model.device)
    reward = torch.tensor(reward, dtype=torch.float, device = model.device)
    done = torch.tensor(done, dtype=torch.uint8)  # Boolean
    weights = torch.tensor(weights, dtype=torch.float, device = model.device)
    #batch_idxes = torch.tensor(batch_idxes, dtype = torch.int64, device = model.device)
    
     #-------------------------------------------------------------------------------------------------------------
    
    # compute the q value
    q_val = compute_q_val(model, state, action)
    
    with torch.no_grad():  # Don't compute gradient info for the target (semi-gradient)
        target = compute_target(model, reward, next_state, done, discount_factor)
    
    # loss is measured from error between current and newly expected Q values
    loss = F.smooth_l1_loss(q_val, target)
    
    # backpropagation of loss to Neural Network (PyTorch magic)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    #-------------------------------------------------------------------------------------------------------------
    mean_loss_value = loss.item() 
    
    with torch.no_grad():
        td_errors = F.smooth_l1_loss(q_val, target,reduction="none") #TD errors, 
    #td_errors = train(obses_t, actions, rewards, obses_tp1, dones, weights)
        if model.device == "cpu":
            td_errors = td_errors.detach().numpy()
        else:
            td_errors = td_errors.cpu().detach().numpy()
    new_priorities = np.abs(td_errors) + prioritized_replay_eps
    #new_priorities = np.abs(loss_value) + prioritized_replay_eps
    memory.update_priorities(batch_idxes, new_priorities)
    #-------------------------------------------------------------------------------------------------------------
    
    return mean_loss_value  # Returns a Python scalar, and releases history (similar to .detach())


#### Selective Experience Replay

In [12]:
class SelectiveReplayMemory:
    
    def __init__(self, fifo_capacity, episodic_memory_capacity):
        self.fifo_capacity = fifo_capacity
        self.episodic_capacity = episodic_memory_capacity
        self.fifo_memory = []
        self.episodic_memory = []
        self.name = "selective"

    def push(self, transition):
        
        if len(self.fifo_memory) == self.fifo_capacity:
            self.fifo_memory = self.fifo_memory[1:]   
        
        self.fifo_memory.append(transition)
    
        #Distribution matching selection
        rank_value = np.random.standard_normal(1)
        self.episodic_memory.append( (transition,rank_value) )
        
        if len(self.episodic_memory) > self.episodic_capacity:
            sorted_episodic = sorted(self.episodic_memory, key=operator.itemgetter(1))
            self.episodic_memory = sorted_episodic[1:]    #Smallest value is index 0 to be removed
        
        #Since no dictionairy was/could be used, duplicate states can be present in the episodic memory
        #Or else could use dictionairy of state to rank value and dict state to other details of transition.

    def sample(self, batch_size):
        
        #50% chance to sample from either buffer?
        #samples = random.sample(self.memory,batch_size)
        #samples = random.sample(self.episodic_memory.keys(),batch_size)
        
        #Or concatenate and sample
        concat = self.fifo_memory + [i[0] for i in self.episodic_memory]
        samples = random.sample(concat,batch_size)
        
        return samples

    def __len__(self):
        return len(self.fifo_memory) #+ len(episodic_memory) #Do we count both memories as len? 

#### Hindsight Experience Replay

In [14]:
class Epsilon_Greedy_Goal(nn.Module):
    
    def __init__(self,device,num_input,num_hidden,num_output):
        nn.Module.__init__(self)
        self.device = device
        self.l1 = nn.Linear(num_input, num_hidden)
        self.l2 = nn.Linear(num_hidden, num_output)
        self.input_dim = num_input  #2 times the state size
        self.hidden_dim = num_hidden
        self.output_dim = num_output
        
    def forward(self,model,state,epsilon,goal):
    
        with torch.no_grad():
            
            if np.random.rand() < epsilon:
                
                combined = np.concatenate(state,goal)
                if not torch.is_tensor(combined):
                    combined_tensor = torch.Tensor(combined)

                combined_tensor = combined_tensor.to(self.device)

                forward_pass = nn.Sequential(
                    self.l1,
                    nn.ReLU(),
                    self.l2
                )
                
                output = forward_pass(combined_tensor)
                a_probs = F.softmax(output)
                
                a = np.random.choice(len(a_probs), size=1,p=a_probs)
            else:
                Q_approx = model(state)
                a = torch.argmax(Q_approx).item()
        return a
    
class HindsightReplayMemory:
    
    def __init__(self, capacity, k,env_name):
        self.capacity = capacity
        self.memory = []
        self.k = k
        self.name = "hindsight"
        self.goals = []
        self.env_name = env_name
        self.current_episode = []
            
        if env_name == "Acrobot-v1":
            self.true_goal_func = lambda s: bool(-np.cos(s[0]) - np.cos(s[1] + s[0]) > 1.)
            self.goal_func = lambda s1,s2: s1 == s2
        elif env_name == "MountainCar-v0":
            self.true_goal_func = lambda s: s[0] >= 0.5 
            self.goal_func = lambda s1,s2: s1[0] == s2[0]
        
    def compute_reward(observed_state,goal):
        
        if goal_func(observed_state,goal):
            return 0
        
        return -1

    def push(self, transition):
                
        if len(self.memory) == self.capacity:
            self.memory = self.memory[1:]   
        
        self.memory.append(transition)
        self.current_episode.append(transition)
        #goals = 
            
    def add_future_goals(self):
        #reward = compute_reward(transition[0],goal)
        for i in range(len(self.current_episode)):
            current_experience = self.current_episode[i]
            observed_state = current_experience[3]
    
            sampled_trans = random.sample(self.current_episode[i+1:],self.k)
            new_rewards = [compute_reward(observed_state,trans[3]) for trans in sampled_trans]
            for new_r in new_rewards:
                current_experience[2] = new_r
                self.memory.append(current_experience)

    def sample(self, batch_size):
        random.sample(self.memory,batch_size)
        
        return samples

    def __len__(self):
        return len(self.memory)

def train_hindsight(model, memory, optimizer, batch_size, discount_factor):
    # DO NOT MODIFY THIS FUNCTION
    
    # don't learn without some decent experience
    if len(memory) < batch_size:
        return None

    # random transition batch is taken from experience replay memory
    transitions = memory.sample(batch_size)
    
    # transition is a list of 4-tuples, instead we want 4 vectors (as torch.Tensor's)
    state, action, reward, next_state, done = zip(*transitions)
    
    # convert to PyTorch and define types
    state = torch.tensor(state, dtype=torch.float, device = model.device)
    action = torch.tensor(action, dtype=torch.int64, device = model.device)  # Need 64 bit to use them as index
    next_state = torch.tensor(next_state, dtype=torch.float, device = model.device)
    reward = torch.tensor(reward, dtype=torch.float, device = model.device)
    done = torch.tensor(done, dtype=torch.uint8)  # Boolean
    
    # compute the q value
    q_val = compute_q_val(model, state, action)
    
    with torch.no_grad():  # Don't compute gradient info for the target (semi-gradient)
        target = compute_target(model, reward, next_state, done, discount_factor)
    
    # loss is measured from error between current and newly expected Q values
    loss = F.smooth_l1_loss(q_val, target)

    # backpropagation of loss to Neural Network (PyTorch magic)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item()  # Returns a Python scalar, and releases history (similar to .detach())

### Training function

In [15]:
def compute_q_val(model, state, action):
    # YOUR CODE HERE
    Q_approx = model(state)
    action_values = torch.gather(Q_approx, dim=1, index=action.reshape(-1,1))
    
    return action_values
    
def compute_target(model, reward, next_state, done, discount_factor):
    # done is a boolean (vector) that indicates if next_state is terminal (episode is done)
    # YOUR CODE HERE

    Q_approx = model(next_state)
    max_Q = torch.max(Q_approx,dim=1)[0]
    target = reward + discount_factor * max_Q

    indices = torch.tensor(np.where(done),dtype=torch.long, device = model.device)
    target = target.scatter(0, indices.reshape(-1), 0)
    target = target.reshape(-1,1)
        
    return target

def train(model, memory, optimizer, batch_size, discount_factor):
    # DO NOT MODIFY THIS FUNCTION
    
    # don't learn without some decent experience
    if len(memory) < batch_size:
        return None

    # random transition batch is taken from experience replay memory
    transitions = memory.sample(batch_size)
    
    # transition is a list of 4-tuples, instead we want 4 vectors (as torch.Tensor's)
    state, action, reward, next_state, done = zip(*transitions)
    
    # convert to PyTorch and define types
    state = torch.tensor(state, dtype=torch.float, device = model.device)
    action = torch.tensor(action, dtype=torch.int64, device = model.device)  # Need 64 bit to use them as index
    next_state = torch.tensor(next_state, dtype=torch.float, device = model.device)
    reward = torch.tensor(reward, dtype=torch.float, device = model.device)
    done = torch.tensor(done, dtype=torch.uint8)  # Boolean
    
    # compute the q value
    q_val = compute_q_val(model, state, action)
    
    with torch.no_grad():  # Don't compute gradient info for the target (semi-gradient)
        target = compute_target(model, reward, next_state, done, discount_factor)
    
    # loss is measured from error between current and newly expected Q values
    loss = F.smooth_l1_loss(q_val, target)

    # backpropagation of loss to Neural Network (PyTorch magic)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item()  # Returns a Python scalar, and releases history (similar to .detach())

In [16]:
def run_episodes(train,select_action_func, model, memory, env, num_episodes, batch_size, discount_factor, learn_rate, 
                 beta_max_iter = 1000,beta_start = 0.4,beta_end = 1.0):
    
    optimizer = optim.Adam(model.parameters(), learn_rate)
    
    global_steps = 0  # Count the steps (do not reset at episode start, to compute epsilon)
    episode_durations = []  #
    
    for i in tqdm(range(num_episodes)):
        # YOUR CODE HERE
        s = env.reset()
        done = False
        local_steps = 0
        
        while not done:
            epsilon = get_epsilon(global_steps)
            #if memory.name == " hindsight":
            #    goal = ..
            #    a = select_action_func(model, s, epsilon,goal)
            #else:
            a = select_action_func(model, s, epsilon)
            observation,reward,done,info = env.step(a)
            
            global_steps += 1
            local_steps += 1
            
            if memory.name == "prioritized":
                memory.add(s, a, reward, observation, float(done))
                beta = parameter_schedule(global_steps,beta_max_iter,beta_start,beta_end)
                loss = train(model, memory, optimizer, batch_size, discount_factor,beta)
            #elif memory.name == "hindsight":
            #    memory.push((s, a, reward, observation, done),goal) 
            #    loss = train(model, memory, optimizer, batch_size, discount_factor)
            else: #memory.name == "uniform" or memory.name == "selective" or memory.name == "hindsight":
                memory.push((s, a, reward, observation, done))
                loss = train(model, memory, optimizer, batch_size, discount_factor)

            s = observation
        
        episode_durations.append(local_steps)
        
        #Check for convergance to terminate perhaps?

    return episode_durations

### Experiments

#### Set multiple seeds 

In [17]:
# We will seed the algorithm (before initializing QNetwork!) for reproducability

if "cuda" in device:
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed = 42 
random.seed(seed)
torch.manual_seed(seed)
env.seed(seed)
np.random.seed(seed) #Added this seed for numpy, as used in selection action

#### Run it

In [18]:
#env = env_cartpole
env = env_car
#env = env_pen
#env = env_acrobot 

In [19]:
#Hyperparameters
num_episodes = 100
batch_size = 64
discount_factor = 0.8
learn_rate = 1e-3
buffer_size = 10000

#Parameters for network, e.g. hidden dim
num_input = len(env.observation_space.sample())
num_hidden = 128
num_output = env.action_space.n

#Parameters for schedule of beta and alpha for prioritized replay
beta_max_iter = 1000
beta_start = 0.4
beta_end = 1.0
prioritized_replay_alpha = 0.6
#prioritized_replay_eps=1e-6 Set default in the training function

#Parameters for selective
episodic_buffer_size = 12000

#Parameters for hindsight
num_policy_hidden = 128
k = 4
env_name = env.unwrapped.spec.id
bigger_buffer_size = 10000 * k

#Picking the memory
memory_method = 3
if memory_method == 1:  #Uniform
    memory = ReplayMemory(buffer_size)
    train_func = train #Uniform
    select_action_func = select_action #epsilon greedy
    
elif memory_method == 2:   #Prioritzed
    memory = PrioritizedReplayBuffer(buffer_size, alpha=prioritized_replay_alpha)
    train_func = train_prioritized #Prioritized Replay
    select_action_func = select_action #epsilon greedy

elif memory_method == 3:   #Selective
    memory = SelectiveReplayMemory(buffer_size,episodic_buffer_size)
    train_func = train #same way as uniform
    select_action_func = select_action #epsilon greedy

elif memory_method == 4: #Hindsight
    memory = HindsightReplayMemory(bigger_buffer_size,k,env_name)
    train_func = train #same way as uniform
    #select_action_func = Epsilon_Greedy_Goal(device,2*num_input,num_policy_hidden,num_output)
    select_action_func = select_action #epsilon greedy
    

#Initializing model
model = QNetwork(device,num_input,num_hidden,num_output)
model = model.to(device)

#Running
start_time = time.time()
episode_durations = run_episodes(train_func,select_action_func, model, memory, env, num_episodes, batch_size, discount_factor, learn_rate,
                                beta_max_iter,beta_start,beta_end)
end_time = time.time()

print("Total duration time: ", str(end_time - start_time))

  0%|          | 0/100 [00:00<?, ?it/s]

64
128
65
130
66
132
67
134
68
136
69
138
70
140
71
142
72
144
73
146
74
148
75
150
76
152
77
154
78
156
79
158
80
160
81
162
82
164
83
166
84
168
85
170
86
172
87
174
88
176
89
178
90
180
91
182
92
184
93
186
94
188
95
190
96
192
97
194
98
196
99
198
100
200
101
202
102
204
103
206
104
208
105
210
106
212
107
214
108
216
109
218
110
220
111
222
112
224
113
226
114
228
115
230
116
232
117
234
118
236
119
238
120
240
121
242
122
244
123
246
124
248
125
250
126
252
127
254
128
256
129
258
130
260
131
262
132
264
133
266
134
268
135
270
136
272
137
274
138
276
139
278
140
280
141
282
142
284
143
286
144
288
145
290
146
292
147
294
148
296
149
298
150
300
151
302
152
304
153
306
154
308
155
310
156
312
157
314
158
316
159
318
160
320
161
322
162
324
163
326
164
328
165
330
166
332
167
334
168
336
169
338
170
340
171
342
172
344
173
346
174
348
175
350
176
352
177
354
178
356
179
358
180
360
181
362
182
364
183
366
184
368
185
370
186
372
187
374
188
376
189
378
190
380
191
382
192
384
193


  2%|▏         | 2/100 [00:01<00:52,  1.86it/s]

309
618
310
620
311
622
312
624
313
626
314
628
315
630
316
632
317
634
318
636
319
638
320
640
321
642
322
644
323
646
324
648
325
650
326
652
327
654
328
656
329
658
330
660
331
662
332
664
333
666
334
668
335
670
336
672
337
674
338
676
339
678
340
680
341
682
342
684
343
686
344
688
345
690
346
692
347
694
348
696
349
698
350
700
351
702
352
704
353
706
354
708
355
710
356
712
357
714
358
716
359
718
360
720
361
722
362
724
363
726
364
728
365
730
366
732
367
734
368
736
369
738
370
740
371
742
372
744
373
746
374
748
375
750
376
752
377
754
378
756
379
758
380
760
381
762
382
764
383
766
384
768
385
770
386
772
387
774
388
776
389
778
390
780
391
782
392
784
393
786
394
788
395
790
396
792
397
794
398
796
399
798
400
800
401
802
402
804
403
806
404
808
405
810
406
812
407
814
408
816
409
818
410
820
411
822
412
824
413
826
414
828
415
830
416
832
417
834
418
836
419
838
420
840
421
842
422
844
423
846
424
848
425
850
426
852
427
854
428
856
429
858
430
860
431
862
432
864
433
866


  5%|▌         | 5/100 [00:02<00:45,  2.10it/s]

908
1816
909
1818
910
1820
911
1822
912
1824
913
1826
914
1828
915
1830
916
1832
917
1834
918
1836
919
1838
920
1840
921
1842
922
1844
923
1846
924
1848
925
1850
926
1852
927
1854
928
1856
929
1858
930
1860
931
1862
932
1864
933
1866
934
1868
935
1870
936
1872
937
1874
938
1876
939
1878
940
1880
941
1882
942
1884
943
1886
944
1888
945
1890
946
1892
947
1894
948
1896
949
1898
950
1900
951
1902
952
1904
953
1906
954
1908
955
1910
956
1912
957
1914
958
1916
959
1918
960
1920
961
1922
962
1924
963
1926
964
1928
965
1930
966
1932
967
1934
968
1936
969
1938
970
1940
971
1942
972
1944
973
1946
974
1948
975
1950
976
1952
977
1954
978
1956
979
1958
980
1960
981
1962
982
1964
983
1966
984
1968
985
1970
986
1972
987
1974
988
1976
989
1978
990
1980
991
1982
992
1984
993
1986
994
1988
995
1990
996
1992
997
1994
998
1996
999
1998
1000
2000
1001
2002
1002
2004
1003
2006
1004
2008
1005
2010
1006
2012
1007
2014
1008
2016
1009
2018
1010
2020
1011
2022
1012
2024
1013
2026
1014
2028
1015
2030
1016
2032
10

KeyboardInterrupt: 

### Plots and analysis

In [ ]:
# And see the results
def smooth(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

plt.plot(smooth(episode_durations, 10))
plt.title('Episode durations per episode')

### With simulation visualization